In [1]:
import pandas as pd
import re
import math

In [2]:
data = pd.read_csv('../../data/cleaned/maple/state data - julian converted.csv')
df = data.copy()
df = df[df["Period"] == "YEAR"]
df = df.reset_index()
df.drop(axis=1, inplace=True, columns=["Program", "Period", "Week Ending", "Geo Level", "State ANSI", "Ag District", "Ag District Code", "County ANSI", "County", "Zip Code", "Region", "watershed_code", "Watershed", "Commodity", "Domain", "Domain Category", "CV (%)"])
display(df.head(40))

,index,Year,State,Data Item,Value
0,0,2019,VERMONT,"MAPLE SYRUP - LENGTH OF SEASON, AVG, MEASURED ...",34
1,1,2019,VERMONT,MAPLE SYRUP - NUMBER OF TAPS,"6,000,000"
2,2,2019,VERMONT,"MAPLE SYRUP - PRODUCTION, MEASURED IN GALLONS","2,070,000"
3,3,2019,VERMONT,"MAPLE SYRUP - YIELD, MEASURED IN GALLONS / TAP",0.345
4,4,2019,VERMONT,"MAPLE SYRUP, HARVEST - END DATE, AVG, MEASURED...",Apr 15
5,5,2019,VERMONT,"MAPLE SYRUP, HARVEST - END DATE, AVG, MEASURED...",105
6,6,2019,VERMONT,"MAPLE SYRUP, HARVEST - END DATE, MAX, MEASURED...",May 3
7,7,2019,VERMONT,"MAPLE SYRUP, HARVEST - END DATE, MAX, MEASURED...",123
8,8,2019,VERMONT,"MAPLE SYRUP, HARVEST - START DATE, AVG, MEASUR...",Mar 12
9,9,2019,VERMONT,"MAPLE SYRUP, HARVEST - START DATE, AVG, MEASUR...",71


In [3]:
data_columns = ["MAPLE SYRUP - YIELD, MEASURED IN GALLONS / TAP", "MAPLE SYRUP - PRODUCTION, MEASURED IN GALLONS", "MAPLE SYRUP - NUMBER OF TAPS", "MAPLE SYRUP - LENGTH OF SEASON, AVG, MEASURED IN DAYS", "MAPLE SYRUP, HARVEST - END DATE, AVG, MEASURED IN JULIAN DATE", "MAPLE SYRUP, HARVEST - START DATE, AVG, MEASURED IN JULIAN DATE"]
df = df[df["Data Item"].isin(data_columns)]
df = df.reset_index()
display(df["Year"].value_counts().sort_index(ascending=False))

2019    6
2018    6
2017    6
2016    6
2015    6
2014    5
2013    4
2012    3
2011    3
2010    3
2009    3
2008    3
2007    3
2006    2
2005    2
2004    2
2003    2
2002    2
2001    2
2000    1
1999    1
1998    1
1997    1
1996    1
1995    1
1994    1
1993    1
1992    1
Name: Year, dtype: int64

In [ ]:
display(df.head(40))

In [4]:
cleaned_data = pd.DataFrame()
for col in "Year", "State":
    cleaned_data[col.upper()] = ""

for col in data_columns:
    new_col = re.match("^(?:MAPLE SYRUP(?: -|,) )([^,\n]+)", col)
    cleaned_data[new_col.group(1)] = ""

cleaned_data.head()

,YEAR,STATE,YIELD,PRODUCTION,NUMBER OF TAPS,LENGTH OF SEASON,HARVEST - END DATE,HARVEST - START DATE


In [5]:
new_row = dict()

entered = False

for index, row in df.iterrows():

    if not entered:
        new_row = dict()
        new_row["YEAR"] = row["Year"]
        new_row["STATE"] = row["State"]
        entered = True

    if row["Year"] in [2019, 2018, 2017, 2016, 2015]:
        if row["Data Item"] == "MAPLE SYRUP - YIELD, MEASURED IN GALLONS / TAP":
            new_row["YIELD"] = row["Value"]
        elif row["Data Item"] == "MAPLE SYRUP - PRODUCTION, MEASURED IN GALLONS":
            new_row["PRODUCTION"] = row["Value"]
        elif row["Data Item"] == "MAPLE SYRUP - NUMBER OF TAPS":
            new_row["NUMBER OF TAPS"] = row["Value"]
        elif row["Data Item"] == "MAPLE SYRUP - LENGTH OF SEASON, AVG, MEASURED IN DAYS":
            new_row["LENGTH OF SEASON"] = row["Value"]
        elif row["Data Item"] == "MAPLE SYRUP, HARVEST - END DATE, AVG, MEASURED IN JULIAN DATE":
            new_row["HARVEST - END DATE"] = row["Value"]
        elif row["Data Item"] == "MAPLE SYRUP, HARVEST - START DATE, AVG, MEASURED IN JULIAN DATE":
            new_row["HARVEST - START DATE"] = row["Value"]
            cleaned_data = cleaned_data.append(new_row, ignore_index=True)
            entered = False

    elif row["Year"] == 2014:
        if row["Data Item"] == "MAPLE SYRUP - YIELD, MEASURED IN GALLONS / TAP":
            new_row["YIELD"] = row["Value"]
        elif row["Data Item"] == "MAPLE SYRUP - PRODUCTION, MEASURED IN GALLONS":
            new_row["PRODUCTION"] = row["Value"]
        elif row["Data Item"] == "MAPLE SYRUP - NUMBER OF TAPS":
            new_row["NUMBER OF TAPS"] = row["Value"]
        elif row["Data Item"] == "MAPLE SYRUP - LENGTH OF SEASON, AVG, MEASURED IN DAYS":
            new_row["LENGTH OF SEASON"] = row["Value"]
        elif row["Data Item"] == "MAPLE SYRUP, HARVEST - END DATE, AVG, MEASURED IN JULIAN DATE":
            new_row["HARVEST - END DATE"] = row["Value"]
            cleaned_data = cleaned_data.append(new_row, ignore_index=True)
            entered = False

    elif row["Year"] == 2013:
        if row["Data Item"] == "MAPLE SYRUP - LENGTH OF SEASON, AVG, MEASURED IN DAYS":
            new_row["LENGTH OF SEASON"] = row["Value"]
        elif row["Data Item"] == "MAPLE SYRUP - PRODUCTION, MEASURED IN GALLONS":
            new_row["PRODUCTION"] = row["Value"]
        elif row["Data Item"] == "MAPLE SYRUP - NUMBER OF TAPS":
            new_row["NUMBER OF TAPS"] = row["Value"]
        elif row["Data Item"] == "MAPLE SYRUP - YIELD, MEASURED IN GALLONS / TAP":
            new_row["YIELD"] = row["Value"]
            cleaned_data = cleaned_data.append(new_row, ignore_index=True)
            entered = False
    
    elif row["Year"] in [2012, 2011, 2010, 2009, 2008, 2007]:
        if row["Data Item"] == "MAPLE SYRUP - PRODUCTION, MEASURED IN GALLONS":
            new_row["PRODUCTION"] = row["Value"]
        elif row["Data Item"] == "MAPLE SYRUP - NUMBER OF TAPS":
            new_row["NUMBER OF TAPS"] = row["Value"]
        elif row["Data Item"] == "MAPLE SYRUP - YIELD, MEASURED IN GALLONS / TAP":
            new_row["YIELD"] = row["Value"]
            cleaned_data = cleaned_data.append(new_row, ignore_index=True)
            entered = False

    elif row["Year"] in [2006, 2005, 2004, 2003, 2002, 2001]:
        if row["Data Item"] == "MAPLE SYRUP - PRODUCTION, MEASURED IN GALLONS":
            new_row["PRODUCTION"] = row["Value"]
        elif row["Data Item"] == "MAPLE SYRUP - YIELD, MEASURED IN GALLONS / TAP":
            new_row["YIELD"] = row["Value"]
            num_taps = int(round(int(new_row["PRODUCTION"].replace(',', '')) / float(new_row["YIELD"].replace(',', '')), -4))
            num_taps = "{:,d}".format(num_taps)
            new_row["NUMBER OF TAPS"] = num_taps
            cleaned_data = cleaned_data.append(new_row, ignore_index=True)
            entered = False

    else:
        if row["Data Item"] == "MAPLE SYRUP - PRODUCTION, MEASURED IN GALLONS":
            new_row["PRODUCTION"] = row["Value"]
            cleaned_data = cleaned_data.append(new_row, ignore_index=True)
            entered = False



In [ ]:
cleaned_data

In [ ]:
cleaned_data.to_csv("../../data/cleaned/maple/cleaned state data - julian converted.csv")